In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [24]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [25]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [26]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
270,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241]
271,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242]
272,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243]
273,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244]


In [27]:
df_2020 = df[['Title']]

In [28]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [30]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 


In [31]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [32]:
df_2020['id'] = df_2020['Title'].map(lambda x: ids(str(x)))

<ipython-input-32-a06ca3f56d32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['id'] = df_2020['Title'].map(lambda x: ids(str(x)))


In [33]:
df_2020

,Title,id
0,The Grudge,1970.0
1,Underwater,443791.0
2,Like a Boss,526019.0
3,Three Christs,418533.0
4,Inherit the Viper,634904.0
...,...,...
270,We Can Be Heroes,615677.0
271,News of the World,581032.0
272,One Night in Miami...,661914.0
273,Promising Young Woman,582014.0


In [36]:
df_2020[df_2020['id'].isnull()]

,Title,id
54,No films had been released on this date as res...,NaN
126,The SpongeBob Movie: Sponge on the Run (Canadi...,NaN


In [37]:
df_2020= df_2020.drop([54,126])

In [38]:
df_2020['id'] = df_2020['id'].astype('int')

In [39]:
df_2020

,Title,id
0,The Grudge,1970
1,Underwater,443791
2,Like a Boss,526019
3,Three Christs,418533
4,Inherit the Viper,634904
...,...,...
270,We Can Be Heroes,615677
271,News of the World,581032
272,One Night in Miami...,661914
273,Promising Young Woman,582014


In [40]:
def genres(movie_id):
    genres=[]
  
    movie_details = tmdb_movie.details(movie_id)
    movie_genres = movie_details.genres
    if movie_genres:
          genre_str = " " 
          for i in range(0,len(movie_genres)):
              genres.append(movie_genres[i]['name'])
          return genre_str.join(genres)
    else:
          return np.NaN



In [41]:
df_2020['genres'] = df_2020['id'].map(lambda x: genres(int(x)))

In [42]:
df_2020

,Title,id,genres
0,The Grudge,1970,Horror Mystery Thriller
1,Underwater,443791,Action Horror Science Fiction
2,Like a Boss,526019,Comedy
3,Three Christs,418533,Drama
4,Inherit the Viper,634904,Crime Thriller Drama
...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy
271,News of the World,581032,Drama Western Adventure Action
272,One Night in Miami...,661914,Drama
273,Promising Young Woman,582014,Thriller Crime Drama


In [43]:
def keywords(movie_id):
    keywords=[]

    movie_keywords = tmdb_movie.keywords(movie_id)
    #print(movie_keywords)
    if movie_keywords:
          #print(movie_keywords['keywords'])
          return movie_keywords['keywords']
    else:
          return np.NaN

In [44]:
df_2020['keywords'] = df_2020['id'].map(lambda x: keywords(int(x)))

In [45]:
df_2020

,Title,id,genres,keywords
0,The Grudge,1970,Horror Mystery Thriller,"[[id, name], [id, name], [id, name], [id, name..."
1,Underwater,443791,Action Horror Science Fiction,"[[id, name], [id, name], [id, name], [id, name..."
2,Like a Boss,526019,Comedy,"[[id, name], [id, name]]"
3,Three Christs,418533,Drama,[]
4,Inherit the Viper,634904,Crime Thriller Drama,"[[id, name], [id, name], [id, name], [id, name..."
...,...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy,"[[id, name], [id, name]]"
271,News of the World,581032,Drama Western Adventure Action,"[[id, name], [id, name], [id, name], [id, name..."
272,One Night in Miami...,661914,Drama,"[[id, name], [id, name], [id, name], [id, name..."
273,Promising Young Woman,582014,Thriller Crime Drama,"[[id, name], [id, name], [id, name], [id, name..."


In [46]:
df_2020['keywords'] = df_2020['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [47]:
s = df_2020.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

<ipython-input-47-a06db21c5fcc>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df_2020.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


based on novel or book    30
woman director            21
murder                    17
based on true story       14
sequel                    12
Name: keyword, dtype: int64

In [48]:
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [49]:
s = s[s > 1]
stemmer = SnowballStemmer('english')

In [50]:
def keywordsFiltered(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [51]:
df_2020['keywords'] = df_2020['keywords'].apply(keywordsFiltered)
df_2020['keywords'] = df_2020['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [52]:
df_2020

,Title,id,genres,keywords
0,The Grudge,1970,Horror Mystery Thriller,"[nurs, hous, remak, tokyo, japan]"
1,Underwater,443791,Action Horror Science Fiction,"[monster, self sacrific, animal attack, disast..."
2,Like a Boss,526019,Comedy,[]
3,Three Christs,418533,Drama,[]
4,Inherit the Viper,634904,Crime Thriller Drama,"[small town, ohio, drug deal, drug, famili]"
...,...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy,"[superhero, famili]"
271,News of the World,581032,Drama Western Adventure Action,"[based on novel or book, texa, hors, shootout,..."
272,One Night in Miami...,661914,Drama,"[miami, florida, based on play or mus]"
273,Promising Young Woman,582014,Thriller Crime Drama,"[ohio, psychopath, horror, reveng, childhood f..."


In [53]:
df_2020['keywords'] = df_2020['keywords'].map(lambda x: " ".join(x))
df_2020['keywords']

0                           nurs hous remak tokyo, japan
1      monster self sacrific animal attack disast gia...
2                                                       
3                                                       
4                  small town ohio drug deal drug famili
                             ...                        
270                                     superhero famili
271    based on novel or book texa hors shootout orph...
272                  miami, florida based on play or mus
273    ohio psychopath horror reveng childhood friend...
274    loss of loved on boston, massachusett grief lo...
Name: keywords, Length: 273, dtype: object

In [54]:
def credits(movie_id):
    cast=[]
  
    movie_credits = tmdb_movie.credits(movie_id)
    if movie_credits['cast'] and movie_credits['crew']:
            for i in movie_credits['crew']:
                if i.get('job') == 'Director':
                    cast.append(i.get('name'))
                    break
            for i in range(0, len(movie_credits['cast'])):
                cast.append(movie_credits['cast'][i]['name'])
            return cast
    else:
            return np.NaN

In [55]:
df_2020['cast'] = df_2020['id'].map(lambda x: credits(int(x)))

In [56]:
df_2020

,Title,id,genres,keywords,cast
0,The Grudge,1970,Horror Mystery Thriller,"nurs hous remak tokyo, japan","[Takashi Shimizu, Sarah Michelle Gellar, Jason..."
1,Underwater,443791,Action Horror Science Fiction,monster self sacrific animal attack disast gia...,"[William Eubank, Kristen Stewart, Vincent Cass..."
2,Like a Boss,526019,Comedy,,"[Miguel Arteta, Tiffany Haddish, Rose Byrne, S..."
3,Three Christs,418533,Drama,,"[Jon Avnet, Richard Gere, Peter Dinklage, Walt..."
4,Inherit the Viper,634904,Crime Thriller Drama,small town ohio drug deal drug famili,"[Anthony Jerjen, Josh Hartnett, Margarita Levi..."
...,...,...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy,superhero famili,"[Robert Rodriguez, YaYa Gosselin, Lyon Daniels..."
271,News of the World,581032,Drama Western Adventure Action,based on novel or book texa hors shootout orph...,"[Paul Greengrass, Tom Hanks, Helena Zengel, Mi..."
272,One Night in Miami...,661914,Drama,"miami, florida based on play or mus","[Regina King, Kingsley Ben-Adir, Eli Goree, Al..."
273,Promising Young Woman,582014,Thriller Crime Drama,ohio psychopath horror reveng childhood friend...,"[Emerald Fennell, Carey Mulligan, Bo Burnham, ..."


In [57]:
df_2020['cast'] = df_2020['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
df_2020['cast'] = df_2020['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [58]:
df_2020

,Title,id,genres,keywords,cast
0,The Grudge,1970,Horror Mystery Thriller,"nurs hous remak tokyo, japan","[takashishimizu, sarahmichellegellar, jasonbeh..."
1,Underwater,443791,Action Horror Science Fiction,monster self sacrific animal attack disast gia...,"[williameubank, kristenstewart, vincentcassel,..."
2,Like a Boss,526019,Comedy,,"[miguelarteta, tiffanyhaddish, rosebyrne, salm..."
3,Three Christs,418533,Drama,,"[jonavnet, richardgere, peterdinklage, waltong..."
4,Inherit the Viper,634904,Crime Thriller Drama,small town ohio drug deal drug famili,"[anthonyjerjen, joshhartnett, margaritalevieva..."
...,...,...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy,superhero famili,"[robertrodriguez, yayagosselin, lyondaniels, a..."
271,News of the World,581032,Drama Western Adventure Action,based on novel or book texa hors shootout orph...,"[paulgreengrass, tomhanks, helenazengel, micha..."
272,One Night in Miami...,661914,Drama,"miami, florida based on play or mus","[reginaking, kingsleyben-adir, eligoree, aldis..."
273,Promising Young Woman,582014,Thriller Crime Drama,ohio psychopath horror reveng childhood friend...,"[emeraldfennell, careymulligan, boburnham, ali..."


In [59]:
df_2020['director'] = df_2020['cast'].map(lambda x:x[0] if len(x) >=1 else x)

In [60]:
df_2020

,Title,id,genres,keywords,cast,director
0,The Grudge,1970,Horror Mystery Thriller,"nurs hous remak tokyo, japan","[takashishimizu, sarahmichellegellar, jasonbeh...",takashishimizu
1,Underwater,443791,Action Horror Science Fiction,monster self sacrific animal attack disast gia...,"[williameubank, kristenstewart, vincentcassel,...",williameubank
2,Like a Boss,526019,Comedy,,"[miguelarteta, tiffanyhaddish, rosebyrne, salm...",miguelarteta
3,Three Christs,418533,Drama,,"[jonavnet, richardgere, peterdinklage, waltong...",jonavnet
4,Inherit the Viper,634904,Crime Thriller Drama,small town ohio drug deal drug famili,"[anthonyjerjen, joshhartnett, margaritalevieva...",anthonyjerjen
...,...,...,...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy,superhero famili,"[robertrodriguez, yayagosselin, lyondaniels, a...",robertrodriguez
271,News of the World,581032,Drama Western Adventure Action,based on novel or book texa hors shootout orph...,"[paulgreengrass, tomhanks, helenazengel, micha...",paulgreengrass
272,One Night in Miami...,661914,Drama,"miami, florida based on play or mus","[reginaking, kingsleyben-adir, eligoree, aldis...",reginaking
273,Promising Young Woman,582014,Thriller Crime Drama,ohio psychopath horror reveng childhood friend...,"[emeraldfennell, careymulligan, boburnham, ali...",emeraldfennell


In [61]:
df_2020['actor1']=df_2020['cast'].map(lambda x:x[1] if len(x) >=2 else x)
df_2020['actor2']=df_2020['cast'].map(lambda x:x[2] if len(x) >=3 else x)
df_2020['actor3']=df_2020['cast'].map(lambda x:x[3] if len(x) >=4 else x)

In [62]:
df_2020

,Title,id,genres,keywords,cast,director,actor1,actor2,actor3
0,The Grudge,1970,Horror Mystery Thriller,"nurs hous remak tokyo, japan","[takashishimizu, sarahmichellegellar, jasonbeh...",takashishimizu,sarahmichellegellar,jasonbehr,williammapother
1,Underwater,443791,Action Horror Science Fiction,monster self sacrific animal attack disast gia...,"[williameubank, kristenstewart, vincentcassel,...",williameubank,kristenstewart,vincentcassel,mamoudouathie
2,Like a Boss,526019,Comedy,,"[miguelarteta, tiffanyhaddish, rosebyrne, salm...",miguelarteta,tiffanyhaddish,rosebyrne,salmahayek
3,Three Christs,418533,Drama,,"[jonavnet, richardgere, peterdinklage, waltong...",jonavnet,richardgere,peterdinklage,waltongoggins
4,Inherit the Viper,634904,Crime Thriller Drama,small town ohio drug deal drug famili,"[anthonyjerjen, joshhartnett, margaritalevieva...",anthonyjerjen,joshhartnett,margaritalevieva,owenteague
...,...,...,...,...,...,...,...,...,...
270,We Can Be Heroes,615677,Action Fantasy Family Comedy,superhero famili,"[robertrodriguez, yayagosselin, lyondaniels, a...",robertrodriguez,yayagosselin,lyondaniels,andywalken
271,News of the World,581032,Drama Western Adventure Action,based on novel or book texa hors shootout orph...,"[paulgreengrass, tomhanks, helenazengel, micha...",paulgreengrass,tomhanks,helenazengel,michaelangelocovino
272,One Night in Miami...,661914,Drama,"miami, florida based on play or mus","[reginaking, kingsleyben-adir, eligoree, aldis...",reginaking,kingsleyben-adir,eligoree,aldishodge
273,Promising Young Woman,582014,Thriller Crime Drama,ohio psychopath horror reveng childhood friend...,"[emeraldfennell, careymulligan, boburnham, ali...",emeraldfennell,careymulligan,boburnham,alisonbrie


In [63]:
df_2020= df_2020.loc[:,['id','director','actor1','actor2','actor3','genres','keywords','Title']]

In [64]:
df_2020 = df_2020.rename(columns={'Title':'title'})

In [ ]:
df_2020['title'] = df_2020['title'].str.lower()
df_2020['actor1'] = df_2020['actor1'].str.lower()
df_2020['actor2'] = df_2020['actor2'].str.lower()
df_2020['actor3'] = df_2020['actor3'].str.lower()
# data['director'] = data['director'].str.lower()
df_2020['comb'] = df_2020['genres'] + ' ' + df_2020['keywords'] + ' '  + df_2020['actor1'] + ' '  + df_2020['actor2'] + ' '  + df_2020['actor3'] + ' ' + df_2020['director']

In [65]:
df_2020

,id,director,actor1,actor2,actor3,genres,keywords,title
0,1970,takashishimizu,sarahmichellegellar,jasonbehr,williammapother,Horror Mystery Thriller,"nurs hous remak tokyo, japan",The Grudge
1,443791,williameubank,kristenstewart,vincentcassel,mamoudouathie,Action Horror Science Fiction,monster self sacrific animal attack disast gia...,Underwater
2,526019,miguelarteta,tiffanyhaddish,rosebyrne,salmahayek,Comedy,,Like a Boss
3,418533,jonavnet,richardgere,peterdinklage,waltongoggins,Drama,,Three Christs
4,634904,anthonyjerjen,joshhartnett,margaritalevieva,owenteague,Crime Thriller Drama,small town ohio drug deal drug famili,Inherit the Viper
...,...,...,...,...,...,...,...,...
270,615677,robertrodriguez,yayagosselin,lyondaniels,andywalken,Action Fantasy Family Comedy,superhero famili,We Can Be Heroes
271,581032,paulgreengrass,tomhanks,helenazengel,michaelangelocovino,Drama Western Adventure Action,based on novel or book texa hors shootout orph...,News of the World
272,661914,reginaking,kingsleyben-adir,eligoree,aldishodge,Drama,"miami, florida based on play or mus",One Night in Miami...
273,582014,emeraldfennell,careymulligan,boburnham,alisonbrie,Thriller Crime Drama,ohio psychopath horror reveng childhood friend...,Promising Young Woman


In [66]:
df_2020.to_csv('movies2020.csv',index=False)

In [67]:
from google.colab import files
files.download("movies2020.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>